<a href="https://colab.research.google.com/github/Vharshitha07/Automatic-Route-Planning-Using-Astar/blob/main/AI_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install folium geopy

In [5]:
import folium
import heapq

# ──────── City coordinates ─────────
# Latitude and longitude of each major city for visualization
coordinates = {
    "Tirupati": (13.6288, 79.4192),
    "Ongole": (15.5057, 80.0499),
    "Vijayawada": (16.5062, 80.6480),
    "Guntur": (16.3067, 80.4365),
    "Rajahmundry": (17.0005, 81.8040),
    "Anakapalli": (17.6913, 83.0033),
    "Araku": (18.3270, 82.8724),
    "Visakhapatnam": (17.6868, 83.2185),
    "Khammam": (17.2473, 80.1514),
    "Nalgonda": (17.0576, 79.2684),
    "Hyderabad": (17.3850, 78.4867),
    "Kurnool": (15.8281, 78.0373)
}

# ──────── Base road network ─────────
# Each connection represents a direct road and its distance (in km)
base_graph = {
    "Tirupati":     [("Ongole", 175),    ("Kurnool", 425)],
    "Ongole":       [("Vijayawada", 130)],
    "Vijayawada":   [("Guntur", 35),     ("Khammam", 215)],
    "Guntur":       [("Rajahmundry", 215), ("Nalgonda", 230), ("Kurnool", 360)],
    "Rajahmundry":  [("Anakapalli", 165), ("Araku", 270)],
    "Anakapalli":   [("Visakhapatnam", 45)],
    "Araku":        [("Visakhapatnam", 115)],
    "Khammam":      [("Hyderabad", 275)],
    "Nalgonda":     [("Hyderabad", 145), ("Kurnool", 315)],
    "Kurnool":      [("Hyderabad", 290)],
    "Visakhapatnam": [],
    "Hyderabad":    []
}

# ──────── Traffic multipliers ─────────
# These represent slowdown factors on specific routes
traffic_factors = {
    ("Tirupati",    "Ongole"):      1.03,
    ("Tirupati",    "Kurnool"):     1.05,
    ("Ongole",      "Vijayawada"):  1.30,
    ("Vijayawada",  "Guntur"):      1.05,
    ("Vijayawada",  "Khammam"):     1.20,
    ("Guntur",      "Rajahmundry"): 1.10,
    ("Guntur",      "Nalgonda"):    1.35,
    ("Guntur",      "Kurnool"):     1.15,
    ("Rajahmundry", "Anakapalli"):  1.25,
    ("Rajahmundry", "Araku"):       1.40,
    ("Anakapalli",  "Visakhapatnam"): 1.05,
    ("Araku",       "Visakhapatnam"): 1.10,
    ("Khammam",     "Hyderabad"):   1.30,
    ("Nalgonda",    "Hyderabad"):   1.15,
    ("Nalgonda",    "Kurnool"):     1.20,
    ("Kurnool",     "Hyderabad"):   1.00
}

# ──────── Blocked roads ─────────
# These roads are unavailable due to conditions like construction or accidents
blocked_roads = {
    ("Vijayawada", "Khammam"),
    ("Khammam",    "Hyderabad")
}

# ──────── Heuristic estimates (straight-line distance to Hyderabad) ─────────
# This is used as h(x) — heuristic cost in A* algorithm
heuristic_dist = {
    "Tirupati": 550, "Ongole": 480, "Vijayawada": 370,
    "Guntur": 390, "Rajahmundry": 440, "Anakapalli": 385,
    "Araku": 300, "Visakhapatnam": 340, "Khammam": 275,
    "Nalgonda": 145, "Kurnool": 290, "Hyderabad": 0
}

# ──────── Average speed assumption ─────────
AVG_SPEED = 60.0  # km/h

# ──────── A* Search Algorithm with Travel Time as Cost ─────────
def a_star_with_time(start, goal):
    open_set = []

    # Initial node: g(x)=0, h(x)=heuristic to goal, f(x)=g+h
    heapq.heappush(open_set, (heuristic_dist[start]/AVG_SPEED, 0.0, start, [start]))
    visited = set()

    while open_set:
        f, g, current, path = heapq.heappop(open_set)  # f = g + h
        if current == goal:
            return path, g

        if current in visited:
            continue
        visited.add(current)

        for neighbor, dist in base_graph[current]:
            # Skip blocked roads
            if (current, neighbor) in blocked_roads or (neighbor, current) in blocked_roads:
                continue

            # Get traffic multiplier
            mult = traffic_factors[(current, neighbor)]

            # ───── g(x): Cost so far (actual time from start) ─────
            travel_time = (dist / AVG_SPEED) * mult
            g2 = g + travel_time

            # ───── h(x): Heuristic estimate to goal ─────
            h2 = heuristic_dist[neighbor] / AVG_SPEED

            # ───── f(x) = g(x) + h(x) ─────
            heapq.heappush(open_set, (g2 + h2, g2, neighbor, path + [neighbor]))

    return None, None  # No path found

# ──────── Run A* Search and Output Results ─────────
start, end = "Tirupati", "Visakhapatnam"
path, total_time = a_star_with_time(start, end)

if path is None:
    print("No viable path found.")
else:
    print(f"Optimal path: {' → '.join(path)}")
    print(f"Total estimated travel time: {total_time:.2f} h\n")
    print("Segment details:")
    for u, v in zip(path, path[1:]):
        dist = next(d for nbr, d in base_graph[u] if nbr == v)
        mult = traffic_factors[(u, v)]
        time_h = (dist / AVG_SPEED) * mult
        print(f"  {u} → {v}: {dist} km, traffic ×{mult}, ≈ {time_h:.2f} h")

# ──────── Visualization with Folium ─────────
m = folium.Map(location=[16.5, 80.5], zoom_start=7)

# Add city markers
for city, coord in coordinates.items():
    folium.Marker(coord, popup=city).add_to(m)

# Add all roads with traffic/blocked status
for city in base_graph:
    for neighbor, dist in base_graph[city]:
        coords = [coordinates[city], coordinates[neighbor]]
        if (city, neighbor) in blocked_roads:
            folium.PolyLine(coords, color='red', dash_array='10', tooltip='Blocked').add_to(m)
        else:
            mult = traffic_factors[(city, neighbor)]
            t = (dist / AVG_SPEED) * mult
            tooltip = f"{city}→{neighbor}: {dist} km, ×{mult}, {t:.2f} h"
            folium.PolyLine(coords, color='gray', weight=2+(mult-1)*10, tooltip=tooltip).add_to(m)

# Highlight the optimal path in green
if path:
    for u, v in zip(path, path[1:]):
        folium.PolyLine([coordinates[u], coordinates[v]],
                        color='green', weight=5,
                        tooltip=f"{u} → {v}").add_to(m)

# Save the map
m.save("andhra_pradesh_time_a_star_full_overrides.html")
print("\nMap saved to andhra_pradesh_time_a_star_full_overrides.html")

Optimal path: Tirupati → Ongole → Vijayawada → Guntur → Rajahmundry → Anakapalli → Visakhapatnam
Total estimated travel time: 14.60 h

Segment details:
  Tirupati → Ongole: 175 km, traffic ×1.03, ≈ 3.00 h
  Ongole → Vijayawada: 130 km, traffic ×1.3, ≈ 2.82 h
  Vijayawada → Guntur: 35 km, traffic ×1.05, ≈ 0.61 h
  Guntur → Rajahmundry: 215 km, traffic ×1.1, ≈ 3.94 h
  Rajahmundry → Anakapalli: 165 km, traffic ×1.25, ≈ 3.44 h
  Anakapalli → Visakhapatnam: 45 km, traffic ×1.05, ≈ 0.79 h

Map saved to andhra_pradesh_time_a_star_full_overrides.html
